In [1]:
import pickle as cPickle
import numpy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "C:/Users/user/Downloads/ud120-projects-master/ud120-projects-master/tools/word_data.pkl", authors_file="C:/Users/user/Downloads/ud120-projects-master/ud120-projects-master/tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features
        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions
        4 objects are returned:
            -- training/testing features
            -- training/testing labels
    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = cPickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [9]:
from time import time

features_train, features_test, labels_train, labels_test = preprocess()
#slicing for faster results
#features_train = features_train[:len(features_train)//100]
#labels_train = labels_train[:len(labels_train)//100]

from sklearn.svm import SVC

clf=SVC(C=10000., kernel='rbf',gamma='scale')
t0=time()

clf.fit(features_train, labels_train)
print("Training time:",round(time()-t0,3),"s")

t1=time()

pred=clf.predict(features_test)

print("Testing time:", round(time()-t1,3),"s")

from sklearn.metrics import accuracy_score

accuracy=accuracy_score(pred, labels_test)
print("Accuracy is equal to %0.4F %%" % (accuracy*100))




no. of Chris training emails: 7936
no. of Sara training emails: 7884
Training time: 229.686 s
Testing time: 22.196 s
Accuracy is equal to 99.6018 %
